# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [16]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
import pandas_datareader.data as pdr
import yfinance as yf
import matplotlib
yf.pdr_override()

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [17]:
stocks =pd.read_csv("constituents.csv")
stocks

,Symbol,Name,Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott,Health Care
3,ABBV,AbbVie,Health Care
4,ACN,Accenture,Information Technology
...,...,...,...
498,YUM,Yum! Brands,Consumer Discretionary
499,ZBRA,Zebra Technologies,Information Technology
500,ZBH,Zimmer Biomet,Health Care
501,ZION,Zions Bancorporation,Financials


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [18]:
# from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [19]:
NGS=['BRK-B', 'BF-B', 'ITW']
symbol=NGS[1]
data=pdr.get_quote_yahoo(symbol)
price=data["price"].to_dict()[symbol]
marketcap=data["marketCap"].to_dict()[symbol]
price,marketcap


(67.07, 32167041024)

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [20]:
columns=["Ticker","Stock Price","Market Capitalization","Number of Shares to Buy"]
final_df=pd.DataFrame([[0,0,0,0]],columns=columns)
final_df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,0,0,0,0


In [21]:
final_df.append(
    pd.Series(
        [symbol,
        price,
        marketcap,
        "N/A"
        ], index=columns
    ),
   
    ignore_index=True
)

/tmp/ipykernel_571/2304564499.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df.append(


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,0,0.00,0,0
1,BF-B,67.07,32167041024,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [22]:
NGS=['BRK-B', 'BF-B', 'ITW']
for stock in NGS:
    symbol=stock
    try:
         data=pdr.get_quote_yahoo(symbol)
    except:
        print(stock)
        continue
    price=data["price"].to_dict()[symbol]
    marketcap=data["marketCap"].to_dict()[symbol]
    final_df=final_df.append(
        pd.Series(
            [
                stock,
                price,
                marketcap,
                "N/A"
            ],index=columns
        ),ignore_index=True
    )


/tmp/ipykernel_571/3384419195.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df=final_df.append(
/tmp/ipykernel_571/3384419195.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df=final_df.append(
/tmp/ipykernel_571/3384419195.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df=final_df.append(


In [23]:
oks=final_df["Ticker"].to_list()
for i in stocks["Symbol"]:
    if i not in oks:
        print(i)
len(stocks["Symbol"])-len(oks)

MMM
AOS
ABT
ABBV
ACN
ATVI
ADM
ADBE
ADP
AAP
AES
AFL
A
APD
AKAM
ALK
ALB
ARE
ALGN
ALLE
LNT
ALL
GOOGL
GOOG
MO
AMZN
AMCR
AMD
AEE
AAL
AEP
AXP
AIG
AMT
AWK
AMP
ABC
AME
AMGN
APH
ADI
ANSS
AON
APA
AAPL
AMAT
APTV
ACGL
ANET
AJG
AIZ
T
ATO
ADSK
AZO
AVB
AVY
BKR
BALL
BAC
BBWI
BAX
BDX
WRB
BRK.B
BBY
BIO
TECH
BIIB
BLK
BK
BA
BKNG
BWA
BXP
BSX
BMY
AVGO
BR
BRO
BF.B
CHRW
CDNS
CZR
CPT
CPB
COF
CAH
KMX
CCL
CARR
CTLT
CAT
CBOE
CBRE
CDW
CE
CNC
CNP
CDAY
CF
CRL
SCHW
CHTR
CVX
CMG
CB
CHD
CI
CINF
CTAS
CSCO
C
CFG
CLX
CME
CMS
CTSH
CL
CMCSA
CMA
CAG
COP
ED
STZ
CEG
COO
CPRT
GLW
CTVA
CSGP
COST
CTRA
CCI
CSX
CMI
CVS
DHI
DHR
DRI
DVA
DAL
XRAY
DVN
DXCM
FANG
DLR
DFS
DISH
DIS
DG
DLTR
D
DPZ
DOV
DOW
DTE
DUK
DD
DXC
EMN
ETN
EBAY
ECL
EIX
EW
EA
ELV
LLY
EMR
ENPH
ETR
EOG
EPAM
EQT
EFX
EQIX
EQR
ESS
ETSY
RE
EVRG
ES
EXC
EXPE
EXPD
EXR
XOM
FFIV
FDS
FAST
FRT
FDX
FITB
FRC
FSLR
FE
FIS
FISV
FLT
FMC
F
FTNT
FTV
FOXA
FOX
BEN
FCX
GRMN
IT
GEN
GNRC
GD
GE
GIS
GM
GPC
GILD
GPN
GL
GS
HAL
HIG
HAS
HCA
PEAK
HSIC
HSY
HES
HPE
HLT
HOLX
HON
HRL
HST
HWM
HPQ
HUM
HBAN
HI

499

In [24]:
for i in range(0,len(stocks["Symbol"])):
    if "." in stocks["Symbol"][i]:
        stocks["Symbol"][i]=stocks["Symbol"][i].replace(".","-")

In [25]:
for i in stocks["Symbol"]:
    if "." in i:
        print(i)

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [26]:
# data=pdr.get_quote_yahoo(['BRK-B','ITW'])
# symbol_groups=chunks(stocks["Symbol"])

In [27]:
def chunks(lst,n):
    for i in range(0,len(lst),n):
        yield lst[i:i+n]
original_group=list(stocks["Symbol"])
symbol_groups=list(chunks(stocks["Symbol"],100))
symbol_groups

[0      MMM
 1      AOS
 2      ABT
 3     ABBV
 4      ACN
       ... 
 95     CDW
 96      CE
 97     CNC
 98     CNP
 99    CDAY
 Name: Symbol, Length: 100, dtype: object,
 100      CF
 101     CRL
 102    SCHW
 103    CHTR
 104     CVX
        ... 
 195    FSLR
 196      FE
 197     FIS
 198    FISV
 199     FLT
 Name: Symbol, Length: 100, dtype: object,
 200     FMC
 201       F
 202    FTNT
 203     FTV
 204    FOXA
        ... 
 295     MAR
 296     MMC
 297     MLM
 298     MAS
 299      MA
 Name: Symbol, Length: 100, dtype: object,
 300    MTCH
 301     MKC
 302     MCD
 303     MCK
 304     MDT
        ... 
 395     RMD
 396     RHI
 397     ROK
 398     ROL
 399     ROP
 Name: Symbol, Length: 100, dtype: object,
 400    ROST
 401     RCL
 402    SPGI
 403     CRM
 404    SBAC
        ... 
 495    WYNN
 496     XEL
 497     XYL
 498     YUM
 499    ZBRA
 Name: Symbol, Length: 100, dtype: object,
 500     ZBH
 501    ZION
 502     ZTS
 Name: Symbol, dtype: object]

In [28]:
# data1=pdr.get_quote_yahoo(symbol_groups[0])
# data2=pdr.get_quote_yahoo(symbol_groups[1])

In [29]:
# data=pd.concat([data1,data2])
# nd=data[["price","marketCap"]]
# nd=nd.reset_index()
# nd["nan"]=np.nan
# nd
# nd.set_axis(columns,axis=1)
# # len(nd)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,MMM,129.51,71585742848,NaN
1,AOS,62.13,9491973120,NaN
2,ABT,113.51,197912641536,NaN
3,ABBV,153.60,271638544384,NaN
4,ACN,282.14,177818173440,NaN
...,...,...,...,...
195,FSLR,180.19,19209334784,NaN
196,FE,42.73,24431005696,NaN
197,FIS,69.75,41388187648,NaN
198,FISV,102.56,65128468480,NaN


In [30]:
columns

['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']

In [32]:
# pdr.get_quote_yahoo(original_group)

In [33]:
final_df=pd.DataFrame(columns=columns)
for i in range(0,len(symbol_groups)):
    data=pdr.get_quote_yahoo(symbol_groups[i])
    td=data[["price","marketCap"]]
    td=td.reset_index()
    td["nan"]=np.nan
    td=td.set_axis(columns,axis=1)
    final_df=pd.concat([final_df,td])

In [34]:
final_df=final_df.reset_index().drop("index",axis=1)

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [35]:
portfolio_size=input("Enter the value of your portfolio:")
try:
    val=float(portfolio_size)
    print(val)
except ValueError:
    print("That's not a number! \nPlease try again:")
    portfolio_size=input("Enter the value of your portfolio:")
    val=float(portfolio_size)


1000000.0


In [36]:
position_size=float(portfolio_size)/len(final_df.index)
position_size

1988.0715705765408

In [37]:
position_size/500

3.9761431411530817

In [38]:
final_df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,MMM,129.51,71585742848,NaN
1,AOS,62.13,9491973120,NaN
2,ABT,113.51,197912641536,NaN
3,ABBV,153.60,271638544384,NaN
4,ACN,282.14,177818173440,NaN
...,...,...,...,...
498,YUM,130.22,38171779072.0,NaN
499,ZBRA,293.70,15695769600.0,NaN
500,ZBH,123.38,25775069184,NaN
501,ZION,51.39,8040633856,NaN


In [40]:
position_size=val/len(final_df.index)
for i in range(0,len(final_df.index)):
    final_df.loc[i,"Number of Shares to Buy"]=math.floor(position_size/float(final_df.loc[i,"Stock Price"]))
# number_of_apple_share=position_size/500
# math.floor(number_of_apple_share)
final_df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,MMM,129.51,71585742848,15.0
1,AOS,62.13,9491973120,31.0
2,ABT,113.51,197912641536,17.0
3,ABBV,153.60,271638544384,12.0
4,ACN,282.14,177818173440,7.0
...,...,...,...,...
498,YUM,130.22,38171779072.0,15.0
499,ZBRA,293.70,15695769600.0,6.0
500,ZBH,123.38,25775069184,16.0
501,ZION,51.39,8040633856,38.0


In [43]:
final_df[final_df["Ticker"]=="AAPL"]

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
44,AAPL,134.76,2143775490048,14.0


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [99]:
writer=pd.ExcelWriter("recommended_trades.xlsx",engine="xlsxwriter")
final_df.to_excel(writer,"recommended_trades",index=False)


### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [100]:
background_color="#0a0a23"
font_color="#ffffff"

string_format=writer.book.add_format(
    {
        "font_color":font_color,
        "bg_color":background_color,
        "border":1
    }
)
dollar_format=writer.book.add_format(
    {
        "num_format":'$0.00',
        "font_color":font_color,
        "bg_color":background_color,
        "border":1
    }
)
integer_format=writer.book.add_format(
    {
        "num_format":'0',
        "font_color":font_color,
        "bg_color":background_color,
        "border":1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [92]:
# writer.sheets['recommended_trades'].set_column("A:A",18,string_format)
# writer.sheets['recommended_trades'].set_column("B:B",18,string_format)
# writer.sheets['recommended_trades'].set_column("C:C",18,string_format)
# writer.sheets['recommended_trades'].set_column("D:D",18,string_format)

# writer.save()
# writer.sheets["recommended_trades"].write('A1',"Ticker",string_format)
# writer.sheets["recommended_trades"].write('B1',"Stock Price",dollar_format)
# writer.sheets["recommended_trades"].write('C1',"Market Capitalization",dollar_format)
# writer.sheets["recommended_trades"].write('D1',"Number of Shares to Buy",integer_format)


0

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [101]:
column_formats={
    'A':["Ticker",string_format],
    'B':["Stock Price",dollar_format],
    'C':["Market Capitalization",dollar_format],
    "D":["Number of Shares to Buy",integer_format]
}
for column in column_formats.keys():
    writer.sheets["recommended_trades"].set_column(f'{column}:{column}',18,column_formats[column][1])
    writer.sheets["recommended_trades"].write(f'{column}1',column_formats[column][0],column_formats[column][1])

/tmp/ipykernel_571/3182595358.py:10: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [69]:
column_formats["A"]

['Ticker', <xlsxwriter.format.Format at 0x7fc513504fa0>]

## Saving Our Excel Output

Saving our Excel file is very easy:

In [ ]:
writer.save()